In [1]:
import pandas as pd
import numpy as np

## Get Some Data

In [2]:
data = pd.read_csv('dataset/df_view_pay_with_month_week_aqi_weather.csv', index_col=[0, 1])
data

view_cnt  pay_cnt  weekday  month  aqi  weather  wind  \
shop_id time_stamp                                                          
1       2015-10-10         0      188        5     10   64      0.0   0.0   
        2015-10-11         0      272        6     10   68      6.0   0.0   
        2015-10-12         0      257        0     10   85      1.0   0.0   
        2015-10-13         0      239        1     10   84      5.0   0.0   
        2015-10-14         0      277        2     10  143      0.0   0.0   
        2015-10-15         0      227        3     10  164      6.0   0.0   
        2015-10-16         0      242        4     10  135      1.0   0.0   
        2015-10-17         0      293        5     10   78      5.0   0.0   
        2015-10-18         0      291        6     10   66      5.0   0.0   
        2015-10-19         0      256        0     10   67      0.0   0.0   
        2015-10-20         0      262        1     10   76      7.0   0.0   
        2015-10-21         0      271        2     10   69      0.0   0.0   
        2015-10-22         0      260        3     10   85      0.0   0.0   
        2015-10-23         0      240        4     10  102      7.0   0.0   
        2015-10-24         0      291        5     10   77      3.0   0.0   
        2015-10-25         0      286        6     10   71      7.0   0.0   
        2015-10-26         0      264        0     10   83     27.0   0.0   
        2015-10-27         0      295        1     10   66      8.0   0.0   
        2015-10-28         0      310        2     10   86     16.0   0.0   
        2015-10-29         0      212        3     10   76      4.0   0.0   
        2015-10-30         0      331        4     10   51      3.0   0.0   
        2015-10-31         0      347        5     10   40     16.0   0.0   
        2015-11-01         0      381        6     11   66      8.0   0.0   
        2015-11-02         0      332        0     11   80      5.0   0.0   
        2015-11-03         0      404        1     11  111      5.0   0.0   
        2015-11-04         0      392        2     11  112      3.0   0.0   
        2015-11-05         0      496        3     11   89      8.0   0.0   
        2015-11-06         0      430        4     11   63     16.0   0.0   
        2015-11-07         0      351        5     11   46      4.0   0.0   
        2015-11-08         0      288        6     11   42      4.0   0.0   
...                      ...      ...      ...    ...  ...      ...   ...   
2000    2016-10-02         4       39        6     10   65      0.0   2.0   
        2016-10-03         3       50        0     10   37      4.0   5.0   
        2016-10-04         8       29        1     10   46      0.0   1.0   
        2016-10-05         3       35        2     10   56      0.0   1.0   
        2016-10-06         4       76        3     10   54      4.0   5.0   
        2016-10-07        11       68        4     10   43      4.0   2.0   
        2016-10-08        16      158        5     10   36      2.0   5.0   
        2016-10-09        24      176        6     10   42      0.0   5.0   
        2016-10-10        35      161        0     10   42      3.0   2.0   
        2016-10-11        24      163        1     10   46      3.0   2.0   
        2016-10-12        33      159        2     10   52      0.0   2.0   
        2016-10-13        35      177        3     10   67      0.0   2.0   
        2016-10-14        14      148        4     10   58      2.0   2.0   
        2016-10-15         9       78        5     10   55      2.0   1.0   
        2016-10-16         4       60        6     10   60      2.0   2.0   
        2016-10-17        16      163        0     10   80      2.0   2.0   
        2016-10-18        18      157        1     10   61      2.0   1.0   
        2016-10-19        14      164        2     10   50      2.0   1.0   
        2016-10-20        35      165        3     10   40      4.0   2.0   
        2016-10-

In [3]:
def data_generator(config, df):
    batch_size = config.batch_size
    num_steps = config.num_steps
    ID_list = list(df.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        np.random.shuffle(ID_list)
        batch_ID_list = ID_list[0:batch_size]
        batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        for ID in batch_ID_list:
            each = df.loc[ID]
            each = each.reset_index(drop=True)
            _m = each.shape[0] - 1
            assert num_steps < _m, 'lower num_steps!'
            _s_idx = np.random.randint(0, _m-num_steps, 1)[0]
            _e_idx = _s_idx + num_steps
            f1 = each.loc[_s_idx:_e_idx-1].values
            _features.append(f1)
            _labels.append(each.loc[_s_idx+1:_e_idx, 'pay_cnt'].values.reshape(-1, 1))
        
        features = np.stack(_features)
        labels = np.stack(_labels)
        yield features, labels, batch_ID_list   
    

In [4]:
class TrainConfig(object):
    batch_size = 10
    num_steps = 30
    feature_size = 8
    lstm_size = 70
train_config = TrainConfig()

class TestConfig(TrainConfig):
    num_steps = 1
test_config = TestConfig()

In [5]:
f, l, id_list = data_generator(train_config, data).next()
print(f.shape)
print(l.shape)
print(id_list)

(10, 30, 8)
(10, 30, 1)
[891, 43, 483, 674, 1110, 1210, 1872, 1109, 13, 344]


## Play Around With TensoFLow

In [6]:
import tensorflow as tf

In [7]:
# tf.InteractiveSession()

In [8]:
config = train_config

In [26]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, train_config.num_steps, train_config.feature_size])

# cell = tf.contrib.rnn.BasicLSTMCell(config.lstm_size)
cell = tf.contrib.rnn.LSTMCell(config.lstm_size)
initial_state = cell.zero_state(config.batch_size, tf.float32)


## dynamic rnn
# outputs, state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state, time_major=False)
# outputs = outputs
# final_state = state

## static rnn
# Permuting batch_size and n_steps
_x = tf.transpose(x, [1, 0, 2])
# Reshaping to (n_steps*batch_size, n_input)
_x = tf.reshape(_x, [-1, train_config.feature_size])
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
_x = tf.split(_x, train_config.num_steps, 0)
outputs, state = tf.contrib.rnn.static_rnn(cell, _x, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
final_state = state



> NOTE: **dynamic rnn** prefered

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    inputs, _, _ = data_generator(train_config, data).next()
    print('inputs.shape: [batch_size, num_steps, feature_size] = {}'.format(inputs.shape))
    output, state = sess.run([outputs, final_state], feed_dict={x: inputs})
    print('output.shape: [batch_size, num_steps, lstm_size] = {}'.format(output.shape))
    print('''final_state tuple(c_state, m_state),
          c or m_state.shape = [batch_size, lstm_size] = {}'''.format(state[1].shape))
#     last_out = output[:, -1, :]
#     print last_out.shape
#     print (last_out == state[1])

inputs.shape: [batch_size, num_steps, feature_size] = (10, 30, 8)
output.shape: [batch_size, num_steps, lstm_size] = (10, 30, 70)
final_state tuple(c_state, m_state),
          c or m_state.shape = [batch_size, lstm_size] = (10, 70)


In [ ]:
# tf.contrib.rnn.static_rnn()